In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, Trainer, TrainingArguments
import torch
from transformers import RobertaConfig, RobertaModel
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from sklearn.metrics import f1_score
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
import torch.nn.functional as F

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
device

device(type='cuda')

In [4]:
df = pd.read_pickle('AnnoMI-full-with-audio-cleaned-text.pkl')

In [5]:
# make a dataset where intelocutor is client

data_client = df[df['interlocutor'] == 'client']

data_therapist = df[df['interlocutor'] == 'therapist']

In [6]:
# Extract unique class names for client and therapist tasks
client_class_names = df['client_talk_type'].dropna().unique().tolist()
therapist_class_names = df['main_therapist_behaviour'].dropna().unique().tolist()

client_class_names, therapist_class_names

(['neutral', 'change', 'sustain'],
 ['question', 'therapist_input', 'reflection', 'other'])

In [7]:
train_data_client, val_data_client = train_test_split(data_client, test_size=0.2, random_state=42)
train_data_therapist, val_data_therapist = train_test_split(data_therapist, test_size=0.2, random_state=42)

In [8]:
tokenizer_twitter_sentiment = AutoTokenizer.from_pretrained('cardiffnlp/twitter-roberta-base-sentiment-latest')
tokenizer_distilbert = AutoTokenizer.from_pretrained('distilbert-base-uncased')
tokenizer_goemotion = AutoTokenizer.from_pretrained('SamLowe/roberta-base-go_emotions')
tokenizer_distilroberta_emo_base = AutoTokenizer.from_pretrained("j-hartmann/emotion-english-distilroberta-base")
tokernizer_roberta_emo_large = AutoTokenizer.from_pretrained("j-hartmann/emotion-english-roberta-large")
tokenizer_roberta_large = AutoTokenizer.from_pretrained("roberta-large")
tokenizer_distilroberta = AutoTokenizer.from_pretrained("distilroberta-base")

In [9]:
tokenizer = tokenizer_roberta_large

In [10]:
train_encodings_client = tokenizer(train_data_client['utterance_text'].tolist(), truncation=True, padding=True)
val_encodings_client = tokenizer(val_data_client['utterance_text'].tolist(), truncation=True, padding=True)
train_encodings_therapist = tokenizer(train_data_therapist['utterance_text'].tolist(), truncation=True, padding=True)
val_encodings_therapist = tokenizer(val_data_therapist['utterance_text'].tolist(), truncation=True, padding=True)

In [11]:
# 4. Dataset Definition
class MTLDataset(Dataset):
    def __init__(self, client_encodings, therapist_encodings, client_labels, therapist_labels):
        self.client_encodings = client_encodings
        self.therapist_encodings = therapist_encodings
        self.client_labels = client_labels
        self.therapist_labels = therapist_labels

    def __getitem__(self, idx):
        client_item = {key: torch.tensor(val[idx]) for key, val in self.client_encodings.items()}
        therapist_item = {key: torch.tensor(val[idx]) for key, val in self.therapist_encodings.items()}
        client_label = torch.tensor(self.client_labels[idx], dtype=torch.long)
        therapist_label = torch.tensor(self.therapist_labels[idx], dtype=torch.long)
        return client_item, therapist_item, client_label, therapist_label

    def __len__(self):
        return len(self.client_labels)

In [12]:
# Convert string labels to categorical integer values
train_data_client['client_talk_type'] = train_data_client['client_talk_type'].astype('category').cat.codes
val_data_client['client_talk_type'] = val_data_client['client_talk_type'].astype('category').cat.codes

train_data_therapist['main_therapist_behaviour'] = train_data_therapist['main_therapist_behaviour'].astype('category').cat.codes
val_data_therapist['main_therapist_behaviour'] = val_data_therapist['main_therapist_behaviour'].astype('category').cat.codes

In [13]:
train_dataset = MTLDataset(train_encodings_client, train_encodings_therapist, train_data_client['client_talk_type'].to_numpy(), train_data_therapist['main_therapist_behaviour'].to_numpy())
val_dataset = MTLDataset(val_encodings_client, val_encodings_therapist, val_data_client['client_talk_type'].to_numpy(), val_data_therapist['main_therapist_behaviour'].to_numpy())

In [14]:
# 6. Model Definition
class MTLModel(nn.Module):
    def __init__(self, base_model_name, num_classes_client, num_classes_therapist):
        super(MTLModel, self).__init__()
        self.shared = RobertaModel.from_pretrained(base_model_name)
        self.client_classifier = nn.Linear(2 * self.shared.config.hidden_size, num_classes_client)
        self.therapist_classifier = nn.Linear(2 * self.shared.config.hidden_size, num_classes_therapist)
        self.attention_weights = nn.Linear(self.shared.config.hidden_size, self.shared.config.hidden_size)
    
    def forward(self, client_input_ids, client_attention_mask, therapist_input_ids, therapist_attention_mask, task_name, use_attention=True):
        client_output = self.shared(input_ids=client_input_ids, attention_mask=client_attention_mask)
        therapist_output = self.shared(input_ids=therapist_input_ids, attention_mask=therapist_attention_mask)
        client_pooled = client_output[0][:, 0, :]
        therapist_pooled = therapist_output[0][:, 0, :]
        
        if use_attention:
            attention_logits = client_pooled @ self.attention_weights(therapist_pooled).T
            attention_scores = F.softmax(attention_logits, dim=1)
            weighted_therapist = attention_scores @ therapist_pooled
            joint_embedding = torch.cat([client_pooled, weighted_therapist], dim=1)
        else:
            joint_embedding = torch.cat([client_pooled, therapist_pooled], dim=1)
        
        if task_name == "client":
            return self.client_classifier(joint_embedding)
        elif task_name == "therapist":
            return self.therapist_classifier(joint_embedding)
        
class MTLModelSmall(nn.Module):
    def __init__(self, base_model_name, num_classes_client, num_classes_therapist):
        super(MTLModelSmall, self).__init__()
        self.shared = RobertaModel.from_pretrained(base_model_name)
        
        # Update the classifier dimensions
        self.client_classifier = nn.Linear(self.shared.config.hidden_size, num_classes_client)
        self.therapist_classifier = nn.Linear(self.shared.config.hidden_size, num_classes_therapist)
        self.attention_weights = nn.Linear(self.shared.config.hidden_size, self.shared.config.hidden_size)
    
    def forward(self, client_input_ids, client_attention_mask, therapist_input_ids, therapist_attention_mask, task_name, use_attention=True):
        client_output = self.shared(input_ids=client_input_ids, attention_mask=client_attention_mask)
        therapist_output = self.shared(input_ids=therapist_input_ids, attention_mask=therapist_attention_mask)
        client_pooled = client_output[0][:, 0, :]
        therapist_pooled = therapist_output[0][:, 0, :]
        
        if use_attention:
            attention_logits = client_pooled @ self.attention_weights(therapist_pooled).T
            attention_scores = F.softmax(attention_logits, dim=1)
            weighted_therapist = attention_scores @ therapist_pooled
            joint_embedding = (client_pooled + weighted_therapist) / 2
        else:
            joint_embedding = (client_pooled + therapist_pooled) / 2
        
        if task_name == "client":
            return self.client_classifier(joint_embedding)
        elif task_name == "therapist":
            return self.therapist_classifier(joint_embedding)
        

class MTLModelWithReverseAttention(nn.Module):
    def __init__(self, base_model_name, num_classes_client, num_classes_therapist):
        super(MTLModelWithReverseAttention, self).__init__()
        self.shared = RobertaModel.from_pretrained(base_model_name)
        self.client_classifier = nn.Linear(2 * self.shared.config.hidden_size, num_classes_client)
        self.therapist_classifier = nn.Linear(2 * self.shared.config.hidden_size, num_classes_therapist)
        
        # Existing attention weights
        self.attention_weights = nn.Linear(self.shared.config.hidden_size, self.shared.config.hidden_size)
        
        # Added reverse attention weights
        self.reverse_attention_weights = nn.Linear(self.shared.config.hidden_size, self.shared.config.hidden_size)
    
    def forward(self, client_input_ids, client_attention_mask, therapist_input_ids, therapist_attention_mask, task_name, use_attention=True):
        client_output = self.shared(input_ids=client_input_ids, attention_mask=client_attention_mask)
        therapist_output = self.shared(input_ids=therapist_input_ids, attention_mask=therapist_attention_mask)
        client_pooled = client_output[0][:, 0, :]
        therapist_pooled = therapist_output[0][:, 0, :]

        if use_attention:
            # Original attention mechanism
            attention_logits = client_pooled @ self.attention_weights(therapist_pooled).T
            attention_scores = F.softmax(attention_logits, dim=1)
            weighted_therapist = attention_scores @ therapist_pooled

            # Reverse attention mechanism
            reverse_attention_logits = therapist_pooled @ self.reverse_attention_weights(client_pooled).T
            reverse_attention_scores = F.softmax(reverse_attention_logits, dim=1)
            weighted_client = reverse_attention_scores @ client_pooled

            # Concatenation
            joint_embedding_client = torch.cat([client_pooled, weighted_therapist], dim=1)
            joint_embedding_therapist = torch.cat([therapist_pooled, weighted_client], dim=1)
        else:
            joint_embedding_client = torch.cat([client_pooled, therapist_pooled], dim=1)
            joint_embedding_therapist = torch.cat([therapist_pooled, client_pooled], dim=1)

        if task_name == "client":
            return self.client_classifier(joint_embedding_client)
        elif task_name == "therapist":
            return self.therapist_classifier(joint_embedding_therapist)
        

class MTLModelWithRegularization(nn.Module):
    def __init__(self, base_model_name, num_classes_client, num_classes_therapist, dropout_rate=0.2):
        super(MTLModelWithRegularization, self).__init__()
        
        self.shared = RobertaModel.from_pretrained(base_model_name)
        
        # Existing attention weights
        self.attention_weights = nn.Linear(self.shared.config.hidden_size, self.shared.config.hidden_size)
        # Added reverse attention weights
        self.reverse_attention_weights = nn.Linear(self.shared.config.hidden_size, self.shared.config.hidden_size)

        # Client Classifier with regularization
        self.client_classifier = nn.Sequential(
            nn.Linear(2 * self.shared.config.hidden_size, self.shared.config.hidden_size),
            nn.BatchNorm1d(num_features=self.shared.config.hidden_size),
            nn.ReLU(),
            nn.Dropout(p=dropout_rate),
            nn.Linear(self.shared.config.hidden_size, num_classes_client)
        )

        # Therapist Classifier with regularization
        self.therapist_classifier = nn.Sequential(
            nn.Linear(2 * self.shared.config.hidden_size, self.shared.config.hidden_size),
            nn.BatchNorm1d(num_features=self.shared.config.hidden_size),
            nn.ReLU(),
            nn.Dropout(p=dropout_rate),
            nn.Linear(self.shared.config.hidden_size, num_classes_therapist)
        )

    def forward(self, client_input_ids, client_attention_mask, therapist_input_ids, therapist_attention_mask, task_name, use_attention=True):
        client_output = self.shared(input_ids=client_input_ids, attention_mask=client_attention_mask)
        therapist_output = self.shared(input_ids=therapist_input_ids, attention_mask=therapist_attention_mask)
        client_pooled = client_output[0][:, 0, :]
        therapist_pooled = therapist_output[0][:, 0, :]

        if use_attention:
            # Original attention mechanism
            attention_logits = client_pooled @ self.attention_weights(therapist_pooled).T
            attention_scores = nn.functional.softmax(attention_logits, dim=1)
            weighted_therapist = attention_scores @ therapist_pooled

            # Reverse attention mechanism
            reverse_attention_logits = therapist_pooled @ self.reverse_attention_weights(client_pooled).T
            reverse_attention_scores = nn.functional.softmax(reverse_attention_logits, dim=1)
            weighted_client = reverse_attention_scores @ client_pooled

            joint_embedding_client = torch.cat([client_pooled, weighted_therapist], dim=1)
            joint_embedding_therapist = torch.cat([therapist_pooled, weighted_client], dim=1)
        else:
            joint_embedding_client = torch.cat([client_pooled, therapist_pooled], dim=1)
            joint_embedding_therapist = torch.cat([therapist_pooled, client_pooled], dim=1)

        if task_name == "client":
            return self.client_classifier(joint_embedding_client)
        elif task_name == "therapist":
            return self.therapist_classifier(joint_embedding_therapist)




In [15]:
NUM_CLASSES_CLIENT = 3
NUM_CLASSES_THERAPIST = 4

epochs = 100
batch_size = 16

model = MTLModelWithRegularization('roberta-large', num_classes_client=NUM_CLASSES_CLIENT, num_classes_therapist=NUM_CLASSES_THERAPIST).to(device)
optimizer = optim.AdamW(model.parameters(), lr=1e-5)
loss_fn = nn.CrossEntropyLoss()

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [17]:
# Lists to store metrics for plotting
training_losses = []
validation_losses = []
training_f1_scores_client = []
training_f1_scores_therapist = []
validation_f1_scores_client = []
validation_f1_scores_therapist = []

best_f1_client = 0.0
best_f1_therapist = 0.0

# Training Loop with tqdm
main_bar = tqdm(range(epochs), desc="Epochs")
for epoch in main_bar:
    model.train()
    total_train_loss = 0
    train_true_labels_client = []
    train_preds_client = []
    train_true_labels_therapist = []
    train_preds_therapist = []

    # tdqm is used to display progress bar
    train_bar = tqdm(train_loader, desc=f"Trainning: Epoch {epoch+1}")
    for client_item, therapist_item, client_label, therapist_label in train_bar:
        optimizer.zero_grad()
        
        # Forward pass for client and therapist tasks
        client_input_ids, client_attention_mask = client_item['input_ids'].to(device), client_item['attention_mask'].to(device)
        therapist_input_ids, therapist_attention_mask = therapist_item['input_ids'].to(device), therapist_item['attention_mask'].to(device)
        client_label, therapist_label = client_label.to(device), therapist_label.to(device)
        
        client_outputs = model(client_input_ids, client_attention_mask, therapist_input_ids, therapist_attention_mask, task_name="client", use_attention=True)
        therapist_outputs = model(client_input_ids, client_attention_mask, therapist_input_ids, therapist_attention_mask, task_name="therapist", use_attention=True)
        
        # Compute loss for both tasks
        client_loss = loss_fn(client_outputs, client_label)
        therapist_loss = loss_fn(therapist_outputs, therapist_label)
        
        # Combine the losses and backpropagate
        combined_loss = client_loss + therapist_loss
        combined_loss.backward()
        optimizer.step()
        total_train_loss += combined_loss.item()
        
        # Store training predictions and true labels
        client_preds = torch.argmax(client_outputs, dim=1)
        therapist_preds = torch.argmax(therapist_outputs, dim=1)
        train_preds_client.extend(client_preds.cpu().numpy())
        train_preds_therapist.extend(therapist_preds.cpu().numpy())
        train_true_labels_client.extend(client_label.cpu().numpy())
        train_true_labels_therapist.extend(therapist_label.cpu().numpy())

    avg_train_loss = total_train_loss / len(train_loader)
    training_losses.append(avg_train_loss)
    
    # Compute training F1-macro scores
    train_client_f1 = f1_score(train_true_labels_client, train_preds_client, average='macro')
    train_therapist_f1 = f1_score(train_true_labels_therapist, train_preds_therapist, average='macro')
    training_f1_scores_client.append(train_client_f1)
    training_f1_scores_therapist.append(train_therapist_f1)

    # Validation Loop for F1-macro evaluation
    model.eval()
    total_val_loss = 0
    all_client_predictions = []
    all_therapist_predictions = []
    all_true_client_labels = []
    all_true_therapist_labels = []
    
    eval_bar = tqdm(val_loader, desc=f"Validation: Epoch {epoch+1}")
    for client_item, therapist_item, client_label, therapist_label in eval_bar:
        with torch.no_grad():
            client_input_ids, client_attention_mask = client_item['input_ids'].to(device), client_item['attention_mask'].to(device)
            therapist_input_ids, therapist_attention_mask = therapist_item['input_ids'].to(device), therapist_item['attention_mask'].to(device)
            client_label, therapist_label = client_label.to(device), therapist_label.to(device)
            
            client_outputs = model(client_input_ids, client_attention_mask, therapist_input_ids, therapist_attention_mask, task_name="client", use_attention=True)
            therapist_outputs = model(client_input_ids, client_attention_mask, therapist_input_ids, therapist_attention_mask, task_name="therapist", use_attention=True)
            
            # Compute loss for both tasks
            client_loss = loss_fn(client_outputs, client_label)
            therapist_loss = loss_fn(therapist_outputs, therapist_label)
            
            # Combine the losses
            combined_loss = client_loss + therapist_loss
            total_val_loss += combined_loss.item()
            
            # Store predictions and true labels for evaluation metrics
            client_preds = torch.argmax(client_outputs, dim=1)
            therapist_preds = torch.argmax(therapist_outputs, dim=1)
            
            all_client_predictions.extend(client_preds.cpu().numpy())
            all_therapist_predictions.extend(therapist_preds.cpu().numpy())
            all_true_client_labels.extend(client_label.cpu().numpy())
            all_true_therapist_labels.extend(therapist_label.cpu().numpy())

    avg_val_loss = total_val_loss / len(val_loader)
    validation_losses.append(avg_val_loss)

    # Compute validation F1-macro scores
    val_client_f1 = f1_score(all_true_client_labels, all_client_predictions, average='macro')
    val_therapist_f1 = f1_score(all_true_therapist_labels, all_therapist_predictions, average='macro')
    validation_f1_scores_client.append(val_client_f1)
    validation_f1_scores_therapist.append(val_therapist_f1)

    # Save the model with the best validation F1-macro score
    if val_client_f1 > best_f1_client and val_therapist_f1 > best_f1_therapist:
        best_f1_client = val_client_f1
        best_f1_therapist = val_therapist_f1
        torch.save(model.state_dict(), 'best_model_client_mtl_2.pth')
        torch.save(model.state_dict(), 'best_model_therapist_mtl_2.pth')
    elif val_client_f1 > best_f1_client:
        best_f1_client = val_client_f1
        torch.save(model.state_dict(), 'best_model_client_mtl_2.pth')
    elif val_therapist_f1 > best_f1_therapist:
        best_f1_therapist = val_therapist_f1
        torch.save(model.state_dict(), 'best_model_therapist_mtl_2.pth')

    # Print metrics for the epoch
    print(f"Epoch {epoch+1} | Training Loss: {avg_train_loss:.4f} | Validation Loss: {avg_val_loss:.4f} \n| Train Client F1: {train_client_f1:.4f} | Train Therapist F1: {train_therapist_f1:.4f} \n| Val Client F1: {val_client_f1:.4f} | Val Therapist F1: {val_therapist_f1:.4f}")

Epochs:   0%|          | 0/100 [00:00<?, ?it/s]

Trainning: Epoch 1:   0%|          | 0/317 [00:00<?, ?it/s]

Validation: Epoch 1:   0%|          | 0/80 [00:00<?, ?it/s]

Epoch 1 | Training Loss: 1.8498 | Validation Loss: 1.4772 
| Train Client F1: 0.4454 | Train Therapist F1: 0.6230 
| Val Client F1: 0.3040 | Val Therapist F1: 0.7576


Trainning: Epoch 2:   0%|          | 0/317 [00:00<?, ?it/s]

...

Validation: Epoch 99:   0%|          | 0/80 [00:00<?, ?it/s]

Epoch 99 | Training Loss: 0.3483 | Validation Loss: 2.7228 
| Train Client F1: 0.8929 | Train Therapist F1: 0.9484 
| Val Client F1: 0.6212 | Val Therapist F1: 0.7850


Trainning: Epoch 100:   0%|          | 0/317 [00:00<?, ?it/s]

Validation: Epoch 100:   0%|          | 0/80 [00:00<?, ?it/s]

Epoch 100 | Training Loss: 0.3475 | Validation Loss: 2.7028 
| Train Client F1: 0.8922 | Train Therapist F1: 0.9501 
| Val Client F1: 0.6148 | Val Therapist F1: 0.7700


In [18]:
epochs_range = range(epochs)

plt.figure(figsize=(20, 12))

# Plot training and validation loss
plt.subplot(2, 2, 1)
plt.plot(epochs_range, training_losses, label='Training Loss')
plt.plot(epochs_range, validation_losses, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')

# Plot training and validation F1 scores for Client
plt.subplot(2, 2, 2)
plt.plot(epochs_range, training_f1_scores_client, label='Training F1 Client')
plt.plot(epochs_range, validation_f1_scores_client, label='Validation F1 Client')
plt.legend(loc='lower right')
plt.title('Training and Validation F1 Score (Client)')

# Plot training and validation F1 scores for Therapist
plt.subplot(2, 2, 3)
plt.plot(epochs_range, training_f1_scores_therapist, label='Training F1 Therapist')
plt.plot(epochs_range, validation_f1_scores_therapist, label='Validation F1 Therapist')
plt.legend(loc='lower right')
plt.title('Training and Validation F1 Score (Therapist)')

plt.tight_layout()
plt.show()

In [19]:
# Final evaluation and classification report

def evaluate_model(model, dataloader, task_name):
    """
    Evaluates the given model on the data from the dataloader and returns true labels and predictions.
    
    Parameters:
    - model: The model to evaluate.
    - dataloader: DataLoader providing the data.
    - task_name: A string, either "client" or "therapist".
    
    Returns:
    - true_labels: A list of true labels.
    - predictions: A list of model's predictions.
    """
    true_labels = []
    predictions = []

    eval_bar = tqdm(dataloader, desc=f"Evaluating: {task_name} task")
    for client_item, therapist_item, client_label, therapist_label in eval_bar:
        with torch.no_grad():
            client_input_ids, client_attention_mask = client_item['input_ids'].to(device), client_item['attention_mask'].to(device)
            therapist_input_ids, therapist_attention_mask = therapist_item['input_ids'].to(device), therapist_item['attention_mask'].to(device)
            client_label, therapist_label = client_label.to(device), therapist_label.to(device)
            
            if task_name == "client":
                outputs = model(client_input_ids, client_attention_mask, therapist_input_ids, therapist_attention_mask, task_name=task_name, use_attention=True)
                preds = torch.argmax(outputs, dim=1)
                true_labels.extend(client_label.cpu().numpy())
            elif task_name == "therapist":
                outputs = model(client_input_ids, client_attention_mask, therapist_input_ids, therapist_attention_mask, task_name=task_name, use_attention=True)
                preds = torch.argmax(outputs, dim=1)
                true_labels.extend(therapist_label.cpu().numpy())
            
            predictions.extend(preds.cpu().numpy())

    return true_labels, predictions


# Load the best models
model_client = model
model_client.load_state_dict(torch.load('best_model_client_mtl_2.pth'))
model_client.eval()

model_therapist = model
model_therapist.load_state_dict(torch.load('best_model_therapist_mtl_2.pth'))
model_therapist.eval()

labels_client, preds_client = evaluate_model(model_client, val_loader, "client")
labels_therapist, preds_therapist = evaluate_model(model_therapist, val_loader, "therapist")

# For client task:
print("\\nFinal Classification Report (Client):")
print(classification_report(labels_client, preds_client, target_names=client_class_names))

# For therapist task:
print("\\nFinal Classification Report (Therapist):")
print(classification_report(labels_therapist, preds_therapist, target_names=therapist_class_names))

Evaluating: client task:   0%|          | 0/80 [00:00<?, ?it/s]

Evaluating: therapist task:   0%|          | 0/80 [00:00<?, ?it/s]

\nFinal Classification Report (Client):
              precision    recall  f1-score   support

     neutral       0.65      0.51      0.57       332
      change       0.75      0.85      0.80       786
     sustain       0.51      0.42      0.46       150

    accuracy                           0.71      1268
   macro avg       0.64      0.59      0.61      1268
weighted avg       0.70      0.71      0.70      1268

\nFinal Classification Report (Therapist):
                 precision    recall  f1-score   support

       question       0.95      0.86      0.90       445
therapist_input       0.84      0.84      0.84       358
     reflection       0.70      0.84      0.76       280
          other       0.72      0.68      0.70       185

       accuracy                           0.82      1268
      macro avg       0.80      0.80      0.80      1268
   weighted avg       0.83      0.82      0.82      1268

